## Pet Store Global Sales Report

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import utils

import re
from os import path

from datetime import datetime
from dateutil.relativedelta import relativedelta

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Input data

In [2]:
# Enter inputs
current_FY_start='2017/4/1' # Current fiscal year start date
report_date='2017/8/31'     # Report end date 

num_yr_to_aggregate = 2     # Number of years to calculate

In [3]:
# Working directory　
filePath='data/'
outputPath='output/'

# Japan sales data 
kdc_thisFY = 'JP_Sales FY2018_gitHub.xlsx' # This FY
kdc_lastFY = 'JP_Sales FY2017_gitHub.xlsx' # Last FY
#kdc_last10FY = 'Sales - Soft・Serv_Prod_FY2010-2016.xlsx' # Last 10 years

# Europe sales data
taDict = dict()
taDict[0] = 'EU_Sales_Report_FY2017_gitHub.xlsx'
taDict[1] = 'EU_Sales_Report_FY201804_gitHub.xlsx'
taDict[2] = 'EU_Sales_Report_FY201805_gitHub.xlsx' 
taDict[3] = 'EU_Sales_Report_FY201806_gitHub.xlsx' 
taDict[4] = 'EU_Sales_Report_FY201807_gitHub.xlsx' 
taDict[5] = 'EU_Sales_Report_FY201808_gitHub.xlsx' 

# Margin and Exchange Rate
ta_margin =  'Europe_margin_rate.xlsx' # TA margin rate

ta_ex_thisFY = 'Exchange Rate_FY2018.xlsx' 
ta_ex_lastFY = 'Exchange Rate_FY2017.xlsx'

In [4]:
current_FY_start = pd.Timestamp(current_FY_start)
last_FY_start = current_FY_start - relativedelta(years = 1)
report_date = pd.Timestamp(report_date)

## Japan Sales Data

In [5]:
# Read in Japan data files
kdc2 = pd.read_excel(path.abspath(filePath + kdc_lastFY),  skiprows=3)
kdc3 = pd.read_excel(path.abspath(filePath + kdc_thisFY),  skiprows=3)

In [6]:
# Concatenate data
kdc_data = pd.concat([kdc2, kdc3])

#print('file 1 (row, col): ', kdc1.shape)
print('file 2 (row, col): ', kdc2.shape)
print('file 3 (row, col): ', kdc3.shape)

print('Number of duplicates to be removed: ', kdc_data[kdc_data.duplicated()].shape[0])
kdc_data.drop_duplicates(inplace=True)
print('Total: (row, col)', kdc_data.shape)

file 2 (row, col):  (80217, 15)
file 3 (row, col):  (35308, 15)
Number of duplicates to be removed:  7520
Total: (row, col) (108005, 15)


In [7]:
# Limit columns 
kdc_data2 = kdc_data[['Year / Month','Company Code', 'Company Name', 'Company Group Name',
                      '3rd Party or Inter',  'Material Name','Sales Quantity',
                      'Actual Sales','Plan Currency' ,'Actual Sales (Ex)' ]].copy()
kdc_data2.head(2)

,Year / Month,Company Code,Company Name,Company Group Name,3rd Party or Inter,Material Name,Sales Quantity,Actual Sales,Plan Currency,Actual Sales (Ex)
0,201604,1001,Alley Cats Pet Shop,08. Pow Production-G1,2,Non target products,33.0,373138.0,JPY,373138.0
1,201604,1001,Alley Cats Pet Shop,08. Pow Production-G1,2,Non target products,15.0,-68318.0,JPY,-68318.0


In [8]:
# Clean up KDC data

print('Shape: ', kdc_data2.shape)

# Delete rows if these columns are null
kdc_data2.dropna(how='all', inplace=True)  
kdc_data2 = kdc_data2[kdc_data2['Material Name'].notnull()]
kdc_data2 = kdc_data2[kdc_data2['Actual Sales'].notnull()]
kdc_data2 = kdc_data2[kdc_data2['Year / Month'].notnull()]
kdc_data2 = kdc_data2[kdc_data2['Company Group Name'].notnull()]

# Limit to 3rd party only
kdc_data2['3rd Party or Inter']=kdc_data2['3rd Party or Inter'].astype(int)
kdc_data2 = kdc_data2[(kdc_data2['3rd Party or Inter'])==3]

# Remove "Production-G and Production-G2". Don't need them.
kdc_data2 = kdc_data2[~kdc_data2['Company Group Name'].str.contains('roduction')]

# Convert to year/month to int
kdc_data2['Year / Month'] = kdc_data2['Year / Month'].astype(int)

# Add 'ExRate' column. This is dummy, not used. For the purpose of matching columns with TA.
kdc_data2['ExRate'] = kdc_data2['Actual Sales (Ex)']/kdc_data2['Actual Sales']

# Reorder columns
cols = ['Year / Month','Company Code', 'Company Name', 'Company Group Name','3rd Party or Inter',  
        'Material Name','Sales Quantity','Actual Sales','Plan Currency','ExRate','Actual Sales (Ex)' ]

kdc_data2 = kdc_data2[cols]

print('Shape: ', kdc_data2.shape)
kdc_data2.head(2)

Shape:  (108005, 10)
Shape:  (71664, 11)


,Year / Month,Company Code,Company Name,Company Group Name,3rd Party or Inter,Material Name,Sales Quantity,Actual Sales,Plan Currency,ExRate,Actual Sales (Ex)
7768,201604,1101,Amazin' Animals Pet Shop,09. Pow Japan,3,Non target products,4.0,21000.0,JPY,1.0,21000.0
7769,201604,1101,Amazin' Animals Pet Shop,09. Pow Japan,3,Non target products,4.0,5460.0,JPY,1.0,5460.0


## TA Data

In [9]:
# Read in this FY TA data

ta_files_to_read = [taDict[k] for k in sorted(taDict.keys()) if len(taDict[k])>0]

ta_data =pd.DataFrame([])
for f in ta_files_to_read:
    temp = pd.read_excel(filePath + f)
    print('Reading ' + f, temp.shape)
    # set column names because it changed in the August file
    temp.columns=['document number', 'customer no', 'Partner', 'booking date', 
                  'Item No','KD Item No', 'description', 'QTY', 'landed price',
                  'total landed price', 'amount piece', 'total amount', 'Date']
    ta_data = ta_data.append(temp, ignore_index = True)  
 
print(ta_data.shape)    

Reading TA_Sales_Report_FY2017_gitHub.xlsx (13078, 13)
Reading TA_Sales_Report_FY201804_gitHub.xlsx (939, 13)
Reading TA_Sales_Report_FY201805_gitHub.xlsx (358, 13)
Reading TA_Sales_Report_FY201806_gitHub.xlsx (445, 13)
Reading TA_Sales_Report_FY201807_gitHub.xlsx (333, 13)
Reading TA_Sales_Report_FY201808_gitHub.xlsx (372, 13)
(15525, 13)


In [10]:
# Remove header cells and blank lines
temp =  ta_data.loc[ta_data['document number'].notnull() & ~ta_data['document number'].isin(['document number'])].copy()
temp.dropna(how ='all', inplace=True)
ta_data2 = temp[temp['Partner'].notnull()]

# Change these to int. They are int in the 201708 file
ta_data2['document number']=pd.to_numeric(ta_data2['document number'], errors='coerce')
ta_data2['customer no']=pd.to_numeric(ta_data2['document number'], errors='coerce')
ta_data2['Item No']=pd.to_numeric(ta_data2['document number'], errors='coerce')

# drop dups from after 2017/05 data only
temp1 = ta_data2[ta_data2.Date<'2017/05'].copy()
temp2 = ta_data2[ta_data2.Date>='2017/05'].copy()
temp2.drop_duplicates(inplace = True)
ta_data2 = pd.concat([temp1, temp2])

print(ta_data2.shape)
ta_data2.head(2)

(15501, 13)


,document number,customer no,Partner,booking date,Item No,KD Item No,description,QTY,landed price,total landed price,amount piece,total amount,Date
0,1101773280,1101773280,Tier GmbH,2016-04-01 00:00:00,1101773280,NaN,Facial Massage Roller for Cats,1,NaN,NaN,47.93,47.93,2016/04
1,1101772220,1101772220,TA Deutschland GmbH,2016-04-01 00:00:00,1101772220,NaN,Facial Massage Roller for Cats,8,NaN,NaN,35.1275,281.02,2016/04


In [11]:
def assign_partner2(st):
    if 'TA CESKA' in st.upper():
        return 'TA Ceska'
    elif 'TA DEUTSCHLAND GMBH' in st.upper():
        return 'TA Deutschland GmbH'
    elif 'TA ÖSTERREICH GMBH' in st.upper():
        return 'TA Österreich GmbH'
    elif 'TA SCHWEIZ AG' in st.upper():
        return 'TA Schweiz AG'
    else:
        return 'Others'

In [12]:
# Add the 'Partner2' column
ta_data2['Partner2'] = ta_data2['Partner'].apply(lambda x: assign_partner2(x))
ta_data2['Group'] ='TA'
ta_data2['Currency']='Euro'

In [13]:
# Prepare to match columns with KDC data

# Reorder columns
col_ta = ['Date', 'customer no', 'Partner', 'Partner2', 'Group','description', 'QTY', 'total amount', 'Currency']
ta_data2 = ta_data2[col_ta]

# Remove '/' from the Date column
ta_data2['Date'] = ta_data2['Date'].str.replace('/', '')

 # Rename the 'Date' column to 'Month' 
ta_data2.columns=['Month', 'customer no', 'Partner', 'Partner2', 'Group','description', 'QTY', 'total amount', 'Currency']

# Change dtype of columns to match KDC_data
ta_data2['Month'] = ta_data2['Month'].astype(int)
ta_data2['customer no'] = ta_data2['customer no'].astype(np.int64)
ta_data2['QTY'] = ta_data2['QTY'].astype(float)
ta_data2['total amount'] = ta_data2['total amount'].astype(float)

ta_data2.head(2)

,Month,customer no,Partner,Partner2,Group,description,QTY,total amount,Currency
0,201604,1101773280,Tier GmbH,Others,TA,Facial Massage Roller for Cats,1.0,47.93,Euro
1,201604,1101772220,TA Deutschland GmbH,TA Deutschland GmbH,TA,Facial Massage Roller for Cats,8.0,281.02,Euro


## TA Exange & Margin Rate   為替とマージン率

In [14]:
# Read in Fx Rate data
temp_fx_thisFY = pd.read_excel(filePath + ta_ex_thisFY, sheetname='Exchange Rate (Actual)', skiprows=3)
temp_fx_lastFY = pd.read_excel(filePath + ta_ex_lastFY, sheetname='Exchange Rate (Actual)', skiprows=3) #index_col='Currency(from)'
margin = pd.read_excel(filePath + ta_margin, sheetname='Sheet1', skiprows=1)  #

In [15]:
# Combine last year and this year's FX rate
fx_thisFY=temp_fx_thisFY.iloc[:, 3:]
fx_lastFY=temp_fx_lastFY.iloc[:, 3:]

fx_thisFY.set_index(['Currency(from)', 'Currency(to)'], inplace=True)
fx_lastFY.set_index(['Currency(from)', 'Currency(to)'], inplace=True)

fx_temp = pd.concat([fx_lastFY, fx_thisFY], axis=1).reset_index()
fx_temp.head(2)

,Currency(from),Currency(to),201604,201605,201606,201607,201608,201609,201610,201611,201612,201701,201702,201703,201704,201705,201706,201707,201708
0,AUD,AUD,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
1,AUD,JPY,84.27,79.58,78.2,78.16,77.22,77.38,79.11,81.58,85.22,85.75,86.62,86.12,83.08,83.44,83.74,87.53,87.06


In [16]:
# Exchange Rate - change the format
fx_rate = fx_temp[(fx_temp['Currency(from)']=='EUR') & (fx_temp['Currency(to)']=='JPY')].iloc[:, 2:].T
fx_rate.columns=['Fx Rate']
fx_rate['Currency']='EUR'
fx_rate['key']=1
fx_rate.reset_index(inplace=True)
fx_rate.columns=['Month', 'Fx Rate', 'Currency', 'key']
fx_rate['Month']=fx_rate['Month'].astype(int)

fx_rate.drop_duplicates(inplace=True)

fx_rate.tail()

,Month,Fx Rate,Currency,key
12,201704,117.95,EUR,1
13,201705,124.10,EUR,1
14,201706,124.52,EUR,1
15,201707,129.42,EUR,1
16,201708,129.88,EUR,1


In [17]:
# Read in margin rate and reformat
# Margin Rate
margin['Partner2'] = margin['Customer Name'].str.replace('その他', 'Others')
margin.drop_duplicates(inplace=True)

margin['key']=1
margin.head()

,Customer Name,マージン率/%,Partner2,key
0,その他,0.0,Others,1
1,TA Ceska,24.8,TA Ceska,1
2,TA Deutschland GmbH,27.6,TA Deutschland GmbH,1
3,TA Österreich GmbH,39.9,TA Österreich GmbH,1
4,TA Schweiz AG,45.8,TA Schweiz AG,1


In [18]:
# Calculate conversion rate =FX/(1-margin rate)
rate = pd.merge(fx_rate, margin, on ='key')

rate = rate[['Month', 'Partner2', 'Currency', 'Fx Rate', 'マージン率/%']]
rate['Conversion Rate'] = rate['Fx Rate'] /(1-rate['マージン率/%']/100) 
rate['Month'] = rate['Month'].astype(int)

# Conversion rate per company
rate2 = rate[['Month', 'Conversion Rate', 'Partner2']]
rate2.tail(5)

,Month,Conversion Rate,Partner2
80,201708,129.880000,Others
81,201708,172.712766,TA Ceska
82,201708,179.392265,TA Deutschland GmbH
83,201708,216.106489,TA Österreich GmbH
84,201708,239.630996,TA Schweiz AG


In [19]:
# Output conversion rate

d=str(report_date.date()).replace('-', '')

output_file = outputPath + r'Europe_conversion_rate_{0}.xlsx'.format(d)
rate.to_excel(output_file, index=False)

## Apply Conversion Rate to TA data  変換レートをTA販売実績に適用

In [20]:
ta_data2.head(2)

,Month,customer no,Partner,Partner2,Group,description,QTY,total amount,Currency
0,201604,1101773280,Tier GmbH,Others,TA,Facial Massage Roller for Cats,1.0,47.93,Euro
1,201604,1101772220,TA Deutschland GmbH,TA Deutschland GmbH,TA,Facial Massage Roller for Cats,8.0,281.02,Euro


In [21]:
# Join TA data and exhange rate
ta_data3 = pd.merge(ta_data2, rate2, how='left', on =['Partner2', 'Month'])
ta_data3.head(2)

,Month,customer no,Partner,Partner2,Group,description,QTY,total amount,Currency,Conversion Rate
0,201604,1101773280,Tier GmbH,Others,TA,Facial Massage Roller for Cats,1.0,47.93,Euro,124.620000
1,201604,1101772220,TA Deutschland GmbH,TA Deutschland GmbH,TA,Facial Massage Roller for Cats,8.0,281.02,Euro,172.127072


In [22]:
# Convert to Japanese Yen
ta_data3['Sales JPY'] = round(ta_data3['total amount']* ta_data3['Conversion Rate'], 3)

In [23]:
# Check for records with Null Ex Rate
ta_data3[ta_data3['Conversion Rate'].isnull()]['Partner2'].unique()

array([], dtype=object)

In [24]:
# Change TA data column name to match KDC column names
col_list = kdc_data2.columns.tolist()
ta_data3.columns = col_list

# Set '3rd Party or Inter'=3 for TA
ta_data3['3rd Party or Inter'] = 3

In [25]:
# Check number of records
print('Number of record KDC', kdc_data2.shape)
print('Number of recor TA', ta_data3.shape)
print('Number of record KDC + TA:', len(kdc_data2) + len(ta_data3))

Number of record KDC (71664, 11)
Number of recor TA (15501, 11)
Number of record KDC + TA: 87165


## Merge KDC and TA data

In [26]:
# Add TA data to KDC data
data_merged = pd.concat([kdc_data2 ,ta_data3])

print('Before: ', data_merged.shape)

# Add a Period column
data_merged['Year-Month'] = pd.to_datetime(data_merged['Year / Month'].astype(str)
                            .apply(lambda x: x[:4]+'-'+x[4:7])).dt.to_period('m')

# Delete records after report_date and limit to 3rd party only
data_merged = data_merged[data_merged['Year-Month'] <= report_date.to_period('m')]
data_merged = data_merged[(data_merged['3rd Party or Inter']==3)]
print('After :', data_merged.shape)

Before:  (87165, 11)
After : (86635, 12)


In [27]:
# Add display company name (clean co name)
# Remove symbols, TA*販社 -> TA
data_merged['Display Co Name'] = data_merged['Company Group Name']
data_merged['Display Co Name'] = data_merged['Display Co Name'].map(lambda x: re.sub('[0-1][0-9].', '',  x))
data_merged['Display Co Name'] = data_merged['Display Co Name'].map(lambda x: re.sub('OCEANIA-G', 'KDAU-G',  x))
data_merged['Display Co Name'] = data_merged['Display Co Name'].map(lambda x: re.sub('TA \D*', 'TA', x))
data_merged['Display Co Name'] = data_merged['Display Co Name'].map(lambda x: re.sub('Others', 'TA',  x))
data_merged['Display Co Name'] = data_merged['Display Co Name'].map(lambda x: re.sub('-G', '',  x))

print('Company Names: ',  data_merged['Display Co Name'].unique())

Company Names:  [' Pow Japan' ' Pow Europe' ' Pow America' ' Pow Oceania' ' Pow Asia'
 ' Pow Canada' 'TA']


In [28]:
# This is dummy. Removed a function for gitHub. Just copy a column
data_merged['Extracted Name'] = data_merged['Material Name']

In [29]:
# Group some products together for business reporting
mapping = [ ('Wobble Wag Giggle Ball', ['Wobble Wag Giggle Ball', 'Pet Zone IQ Treat Ball']),
            ('Pet Bed & Comfort', ['Sneaker Pet Bed','Catit Design Senses Massage Center', 'Suck UK Laptop Cat Scratching Pad', 'K&H Thermo Cat Windowsill Seat', 'KritterWorld Microplush Pet Blanket']),
            ('Holiday & Subscription Box', ['PupBox Holiday Box', 'Barkbox Subscription Box', 'MeowBox Monthly Subscription Box']),
            ('Furbo Dog Camera', ['Furbo Dog Camera', 'Petcube Camera'])
           ] 

d = dict()
for (value, keys) in mapping:
    for key in keys:
        d[key] = value
        
data_merged['対象商材'] = data_merged['Extracted Name'].apply(lambda x: d[x] if x in d else x)

In [30]:
# Limit to reporting items 
report_items = data_merged[data_merged['対象商材'].notnull()]['対象商材'].unique().tolist()
#report_items.remove('Non target products')
report_items=sorted(report_items)

# Limit to requested number of years 
report_start_yr =current_FY_start - relativedelta(years=num_yr_to_aggregate)

standardize_sales_data = data_merged[(data_merged['対象商材'].isin(report_items)) 
                                     & (data_merged['Year-Month']>=report_start_yr.to_period('m'))].copy() 


# Sort
standardize_sales_data.sort_values(by=['対象商材', 'Extracted Name', 'Year-Month', 'Display Co Name'], inplace=True)


standardize_sales_data=standardize_sales_data[['Year-Month', 'Company Code', 'Company Name', 'Company Group Name', 
                                               'Display Co Name','3rd Party or Inter', 
                                               'Material Name', 'Extracted Name', '対象商材', 'Sales Quantity', 
                                               'Actual Sales', 'Plan Currency', 'ExRate', 'Actual Sales (Ex)']]

standardize_sales_data.head(2)

,Year-Month,Company Code,Company Name,Company Group Name,Display Co Name,3rd Party or Inter,Material Name,Extracted Name,対象商材,Sales Quantity,Actual Sales,Plan Currency,ExRate,Actual Sales (Ex)
52363,2016-04,1201,Pets Unleashed,04. Pow America,Pow America,3,Catit Senses 2.0 Circuit,Catit Senses 2.0 Circuit,Catit Senses 2.0 Circuit,4.0,540.0,USD,109.93,59362.2
58373,2016-04,1202,Pets Unlimited,04. Pow America,Pow America,3,Catit Senses 2.0 Circuit,Catit Senses 2.0 Circuit,Catit Senses 2.0 Circuit,2.0,0.0,CAD,NaN,0.0


## Output reconciled sales data for BI tools

In [31]:
# Output non aggregated data for BI tools
d=str(report_date.date()).replace('-', '')
output_file = outputPath + 'Non_Agg_Results_for_BI_tool_{0}.xlsx'.format(d)
standardize_sales_data.to_excel(output_file, sheet_name='Sales Data', index=False)

## Write aggregated data to output (集計結果の出力)

In [32]:
display_cols = ['Year-Month', '3rd Party or Inter', 'Display Co Name', '対象商材', 'Sales Quantity', 'Actual Sales (Ex)']
agg_cols = ['対象商材', '3rd Party or Inter' ,'Year-Month', 'Display Co Name']

agg_sales_last5FY = standardize_sales_data[display_cols].groupby(by=agg_cols).sum()
agg_sales_last5FY['Actual Sales (Ex) in 1000']  = agg_sales_last5FY['Actual Sales (Ex)']/1000

agg_sales_last5FY.head()

Sales Quantity  \
対象商材                     3rd Party or Inter Year-Month Display Co Name                   
Catit Senses 2.0 Circuit 3                  2016-04     Pow America                6.0   
                                                        Pow Europe               384.0   
                                                        Pow Japan                 12.0   
                                                        Pow Oceania                1.0   
                                                       TA                          3.0   

                                                                        Actual Sales (Ex)  \
対象商材                     3rd Party or Inter Year-Month Display Co Name                      
Catit Senses 2.0 Circuit 3                  2016-04     Pow America             59362.200   
                                                        Pow Europe            1873872.290   
                                                        Pow Japan              183000.000   
                                                        Pow Oceania              6151.710   
                                                       TA                       24970.715   

                                                                        Actual Sales (Ex) in 1000  
対象商材                     3rd Party or Inter Year-Month Display Co Name                             
Catit Senses 2.0 Circuit 3                  2016-04     Pow America                     59.362200  
                                                        Pow Europe                    1873.872290  
                                                        Pow Japan                      183.000000  
                                                        Pow Oceania                      6.151710  
                                                       TA                               24.970715

In [33]:
output_file = outputPath + 'Agg_Results_{0}.xlsx'.format(d)
agg_sales_last5FY.to_excel(output_file, sheet_name='Aggregated', index=False)

#output_agg = r'C:\Users\ymatsukawa\Documents\Project\OptionSales\集計結果_Last5FY_{0}_new3.xlsx'.format(str(report_date.date()))
#writer = pd.ExcelWriter(output_agg)

#agg_sales_last5FY.reset_index().to_excel(writer, 'Aggregated', index=False)
#writer.save()

## Plot Graphs

In [34]:
# Create Last FY and This FY's data df and plot charts, save to files

# Convert date to 'Month'-period
report_date_mo = report_date.to_period('m')
current_FY_start_mo = current_FY_start.to_period('m')
last_FY_start_mo = last_FY_start.to_period('m')
last_FY_end_mo = (current_FY_start-relativedelta(months=1)).to_period('m')


reset_data = agg_sales_last5FY.reset_index()

for item in report_items:
    print('Creating a graph for', item)
    
    temp2 = utils.getChartData(reset_data, item, last_FY_start_mo, last_FY_start, report_date)
    
    # This FY data, pivotted (multiple columns for company) for stacked bar graph
    df_thisFY_pivot = utils.get_thisFY_pivot_df(temp2, current_FY_start_mo, report_date_mo)

    # This FY df, NOT pivotted for this FY line graph
    chart_df_thisFY = utils.get_chart_df(temp2, current_FY_start_mo, report_date_mo)

    # Last FY df, NOT pivotted for both bar and line graph
    chart_df_lastFY = utils.get_chart_df(temp2, last_FY_start_mo, last_FY_end_mo)
    
    # Plot chart
    utils.plotChart(df_thisFY_pivot, chart_df_thisFY, chart_df_lastFY, item, report_date, outputPath)


Creating a graph for Catit Senses 2.0 Circuit
Creating a graph for Doggie Drinklets Portable Water Bowl
Creating a graph for Drs. Foster & Smith Whack-a-Mouse Cat Toy
Creating a graph for Facial Massage Roller for Cats
Creating a graph for FireboltCreations Custom Pet Pillow
Creating a graph for Furbo Dog Camera
Creating a graph for H2O4K9 Doggy Water Bottle
Creating a graph for Hammacher Schlemmer Feline's Laser Chasing Scratch Post
Creating a graph for Holiday & Subscription Box
Creating a graph for Kong Rubber Dog Frisbee
Creating a graph for Magic Latch Leash Connection Sytem
Creating a graph for Modkat Litter Box
Creating a graph for Munchiecat Sushi Cat Toys
Creating a graph for Non target products
Creating a graph for Pet Bed & Comfort
Creating a graph for Pet Gear Pup Tub
Creating a graph for Petcube Play
Creating a graph for Pooch Selfie: The Original Dog Selfie Stick
Creating a graph for Siensync Interactive Feeder Dog Bowl
Creating a graph for Sneaker Pet Bed 
Creating a gra